<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/query-expansion/query_expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import collections
import matplotlib.pyplot as plt
%matplotlib inline

import heapq as hp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Introdução

Nesta atividade exercitaremos conceitos sobre expansão de consultas.  Os dados utilizados são de textos de notícias resultados da utilização de técnicas de crawling e scrapping. Os scripts para a coleta desses dados podem ser encontrados [neste](https://github.com/valterlucena/ri_lab_01) repositório.

Primeiramente, vamos importar os dados que serão utilizados.


In [0]:
DATA_URL = 'https://raw.githubusercontent.com/valterlucena/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL).replace(np.nan, '', regex=True)

Agora iremos criar nosso índice invertido, para ser utilizado mais adiante. Utilizaremos a função tokenize da biblioteca NLTK associada à uma expressão regular, que considerará como token apenas as sequências de caracteres não-especiais (com exceção do hífen) ou numéricos que não formem stopwords e quem possuam mais que 2 caracteres.

In [0]:
toker = RegexpTokenizer('''\w+[-']*\w*''')
stop_words = stopwords.words('portuguese')

def isValid(token):
  return token not in stop_words and not bool(re.search(r'\d', token)) and len(token) > 2

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = [token for token in toker.tokenize(document.lower()) if isValid(token)]
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

index = build_index(news.text)

In [0]:
'''
  Calcula o total de documentos que contém um determinado termo
''' 
def count_docs(termo):
  return len(index[termo].keys())

'''
  Calcula o total de documentos que contém os termos a e b
'''
def count_docs_conj(a, b):
  docs_a = index[a].keys()
  docs_b = index[b].keys()
  count = 0
  for doc in docs_a:
    if doc in docs_b:
      count += 1
  return count  

'''
  Retorna o total de documentos na base de dados.
'''
def total_docs():
  return news.text.count()

In [0]:
'''
  Medidas de associação de termos
'''

'''
  Mutal Information Measure
'''
def mutual_information(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  mim = n_ab / (n_a * n_b)
  return mim

'''
  Expected Mutual Information Measure
'''
def expected_mutual_information(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  n = total_docs()
  exp = n * (n_ab / (n_a * n_b))
  emim = n_ab * np.log(exp) if exp != 0 else 0
  return emim

'''
  Pearson's Chi-squared
'''
def chi_square(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  n = total_docs()
  chi = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return chi

'''
  Dice's coefficient
'''
def dice_coefficient(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  dice = n_ab / (n_a + n_b)
  return dice

In [0]:
'''
  Cria uma lista de tuplas contendo o valor de uma determinada métrica para um 
  posting
'''
def build_metric_list(term, metric):
  pairs = []
  for posting in index.keys():
    if posting != termo:
      if metric == 'mim':
        metric_value = mutual_information(termo, posting)
      elif metric == 'emim':
        metric_value = expected_mutual_information(termo, posting)
      elif metric == 'chi-square':
        metric_value = chi_square(termo, posting)
      elif metric == 'dice':
        metric_value = dice_coefficient(termo, posting)
      pairs.append((posting, metric_value))      
  pairs = sorted(pairs, key = lambda x: x[1], reverse=True)
  return pairs

termos = ['presidente', 'lula', 'bolsonaro', 'governo', 'federal']
termos_metricas = {}
for termo in termos:
  metricas = {}
  metricas['mim'] = build_metric_list(termo, 'mim')
  metricas['emim'] = build_metric_list(termo, 'emim')
  metricas['chi-square'] = build_metric_list(termo, 'chi-square')
  metricas['dice'] = build_metric_list(termo, 'dice')
  termos_metricas[termo] = metricas

In [0]:
'''
  Constrói um dataframe com o top 10 de palavras mais associadas à um termo de
  acordo com cada uma das métricas
'''
def build_table(termo):
  mim = [k for k,_ in termos_metricas[termo]['mim'][0:10]]
  emim = [k for k,_ in termos_metricas[termo]['emim'][0:10]]
  chi_square = [k for k,_ in termos_metricas[termo]['chi-square'][0:10]]
  dice = [k for k,_ in termos_metricas[termo]['dice'][0:10]]
  data = {'MIM': mim, 'EMIM': emim, 'X²': chi_square, 'Dice': dice}
  return pd.DataFrame(data)

In [9]:
build_table(termos[0]) # presidente

,MIM,EMIM,X²,Dice
0,requereu,bolsonaro,jair,bolsonaro
1,penal,jair,bolsonaro,brasil
2,especial,brasil,brasil,jair
3,embu,ser,direitos,ser
4,artes,governo,processo,governo
5,nando,direitos,apenas,país
6,moura,país,candidato,direitos
7,injúria,contra,gente,contra
8,difamação,apenas,antes,dia
9,requerida,dia,ser,hoje


In [10]:
build_table(termos[1]) # lula

,MIM,EMIM,X²,Dice
0,elio,ex-presidente,luiz,ex-presidente
1,gaspari,dizer,ex-presidente,dizer
2,resolve,democracia,dizer,democracia
3,pedirá,luiz,inocência,luiz
4,inocência,prisão,inácio,legislação
5,negocio,direitos,legislação,ato
6,trocaria,silva,democracia,silva
7,relativo,dentro,curitiba,dentro
8,conforto,legislação,visão,prisão
9,assemelhariam,ato,condenou,curitiba


In [11]:
build_table(termos[2]) # bolsonaro

,MIM,EMIM,X²,Dice
0,requereu,jair,jair,jair
1,especial,presidente,presidente,presidente
2,embu,ser,onde,ser
3,artes,governo,governo,governo
4,nando,brasil,vai,brasil
5,moura,onde,ser,país
6,injúria,país,antes,onde
7,difamação,vai,brasil,sobre
8,requerida,antes,país,vai
9,promotoria,sobre,deputados,ter


In [12]:
build_table(termos[3]) # governo

,MIM,EMIM,X²,Dice
0,requereu,bolsonaro,reforma,bolsonaro
1,penal,ser,previdência,ser
2,corre,brasil,outra,brasil
3,especial,presidente,justiça,presidente
4,embu,reforma,tudo,todos
5,artes,todos,política,política
6,nando,política,bolsonaro,país
7,moura,federal,falta,contra
8,injúria,justiça,brasil,federal
9,difamação,contra,federal,ter


In [13]:
build_table(termos[4]) # federal

,MIM,EMIM,X²,Dice
0,defendia,ministro,ministro,ministro
1,lobista,justiça,justiça,justiça
2,denunciou,direitos,tribunal,direitos
3,furnas,governo,prisão,governo
4,dino,prisão,direitos,prisão
5,miraglia,ser,fernando,ano
6,comeu,ano,polícia,nacional
7,pão,nacional,humanos,ser
8,amaçou,público,investigações,ter
9,helicóptero,ter,desde,público


In [31]:
'''
  Busca documento-por-vez conjuntiva
'''
def conjuctive_query(query, index, k):
  inverted_lists = []
  results = []
  
  for term in query.split():
    if term in index.keys():
      inverted_list = index[term]
      inverted_lists.append(inverted_list)
  
  documents = [item for inverted_list in inverted_lists for item in inverted_list.items()]
  documents = sorted(documents, key = lambda x: x[0])
    
  for i in range(len(documents)):
    score = 0
    d,f = documents.pop()
    repeat = 1
    for document,freq in documents:
      if document == d:
        score += freq
        repeat += 1
    if score != 0 and repeat == len(inverted_lists):
      score += f
      hp.heappush(results, (score, d))
  return [(d,s) for (s,d) in hp.nlargest(k, results)]

[(117, 124), (64, 18), (123, 9), (50, 9), (98, 2)]

In [32]:
'''
  Expande a consulta com os k termos melhores rankeados para a métrica selecionada
'''
def expanded_query(term, metric, top_k):
  terms = [word for word,_ in termos_metricas[term][metric][0:top_k]]
  new_query = term + ' ' + ' '.join(terms)
  return (new_query, conjuctive_query(new_query, index, 10))

termo_antigo = []
queries = []
results = []
top_k = []
for termo in termos:
  for i in [3, 5, 10]:
    query, result = expanded_query(termo, 'emim', i)
    termo_antigo.append(termo)
    queries.append(query)
    results.append(result)
    top_k.append(i)

pd.options.display.max_colwidth = 160
data = {'Termo': termo_antigo, 'Top k': top_k, 'Consulta Expandida': queries, 'Resultados': results}
pd.DataFrame(data)



,Termo,Top k,Consulta Expandida,Resultados
0,presidente,3,presidente bolsonaro jair brasil,"[(99, 47), (50, 25), (98, 24), (65, 21), (75, 20), (15, 16), (5, 15), (45, 14), (137, 12), (129, 11)]"
1,presidente,5,presidente bolsonaro jair brasil ser governo,"[(99, 58), (98, 41), (50, 38), (75, 31), (45, 25), (111, 19), (5, 19), (137, 18), (94, 12), (10, 11)]"
2,presidente,10,presidente bolsonaro jair brasil ser governo direitos país contra apenas dia,"[(111, 29), (94, 25)]"
3,lula,3,lula ex-presidente dizer democracia,"[(123, 13), (125, 9), (137, 4)]"
4,lula,5,lula ex-presidente dizer democracia luiz prisão,"[(123, 16)]"
5,lula,10,lula ex-presidente dizer democracia luiz prisão direitos silva dentro legislação ato,[]
6,bolsonaro,3,bolsonaro jair presidente ser,"[(99, 32), (98, 21), (75, 21), (111, 16), (45, 16), (15, 16), (50, 15), (37, 14), (105, 13), (137, 12)]"
7,bolsonaro,5,bolsonaro jair presidente ser governo brasil,"[(99, 58), (98, 41), (50, 38), (75, 31), (45, 25), (111, 19), (5, 19), (137, 18), (94, 12), (10, 11)]"
8,bolsonaro,10,bolsonaro jair presidente ser governo brasil onde país vai antes sobre,"[(50, 51)]"
9,governo,3,governo bolsonaro ser brasil,"[(99, 49), (98, 34), (50, 34), (75, 21), (45, 18), (5, 17), (137, 16), (111, 16), (101, 13), (17, 12)]"
